# Pràctica 3: Controlant el robot amb una Xarxa Neuronal

## Introducció

En aquesta pràctica es pretén controlar un robot amb una xarxa neuronal. Per això, entrenarem una xarxa neuronal amb dades de sensors del robot i les comandes que s'han de donar al robot per aconseguir que es mogui cap a l'objectiu.

In [2]:
!pip install aitk.robots aitk.networks tensorflow numpy!pip install aitk.robots aitk.networks tensorflow numpy!pip install aitk aitk.robots

   ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 191.9/475.3 MB 2.8 MB/s eta 0:01:41^C
   ━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 191.9/475.3 MB 2.8 MB/s eta 0:01:41
ERROR: Operation cancelled by user


In [1]:
import random

import aitk.networks as nets
import numpy as np
import aitk.robots as bots

import keras

ModuleNotFoundError: No module named 'aitk'

## Creant la funció per llegir les dades d'entrenament
Crearem una funció que llegeixi les dades d'entrenament i ens retorni una llista de patrons on cada element de la llista és un patró d'entrada i un patró de sortida. A més, ens mostrarà el nombre de patrons llegits.

In [2]:
def carrega_dades_entrenament(nom_fitxer):
    """Retorna les dades d'entrenament"""
    
    data = []
    with open(nom_fitxer, "r") as f:
        for line in f:
            split_line = line.strip().split(" ")
            float_line = [float(x) for x in split_line]
            data.append(float_line)
    print(len(data), " exemples carregats")
    return data

In [3]:
patrons = carrega_dades_entrenament("training_data.txt")

9380  exemples carregats


In [4]:
# Vejam quants paràmetres tenen els patrons
print(len(patrons[0]))

5


In [5]:
# Observem un exemple
print(patrons[0])

[0.5288191487474595, 1.0, 0.0690139683385698, 0.5, 0.0]


## Preprocessament i balanceig de les dades

Les eines de machine learning, incloses les xarxes neuronals, trobaran la manera més directa de reduir l'error. Farem una mica d'anàlisi de les dades que hem recollit. Crearem una funció auxiliar per comptar quin percentatge de les nostres dades d'entrenament fan que el robot es mogui cap endavant, es mogui cap a l'esquerra o es mogui cap a la dreta.

La funció hauria de prendre una llista de patrons com a entrada i retornar una llista de tres valors: el percentatge de patrons amb moviment cap endavant, el percentatge de patrons amb moviment cap a l'esquerra i el percentatge de patrons amb moviment cap a la dreta.

In [6]:
def classifica_moviments(patterns):
    """Retorna una llista de tres valors: 
       - el percentatge de patrons amb moviment cap endavant, 
       - el percentatge de patrons amb moviment cap a l'esquerra i 
       - el percentatge de patrons amb moviment cap a la dreta."""


In [7]:
classifica_moviments(patrons)

forward:  6200
left:  1480
right:  1700


[0.6609808102345416, 0.15778251599147122, 0.1812366737739872]

En la robótica és típic que el robot es mogui més cap a l'objectiu que cap a la dreta o cap a l'esquerra. Per tant, és probable que la xarxa neuronal aprengui a moure's cap a l'objectiu i no a girar. Per això, és important que les dades estiguin balancejades. Per aconseguir-ho, dividirem les dades en tres llistes, una per cada tipus de moviment, i barrejarem les llistes. A continuació, agafarem el mateix nombre de patrons de cada llista i les barrejarem de nou. Això ens donarà un conjunt de dades balancejat.

In [8]:
def balanceja_moviments(patrons):
    """Retorna una llista de patrons amb el mateix nombre d'exemples per a cada tipus de moviment."""


In [9]:
patrons_balancats = balanceja_moviments(patrons)
print(len(patrons_balancats))

4440


Podem intentar entrenar la xarxa neuronal amb les dades sense balancejar i veure com es comporta. Si la xarxa neuronal no aprèn a girar, podem provar de balancejar les dades.

In [ ]:
patrons_entrenament = patrons[:int(len(patrons) * 0.8)]
patrons_validacio = patrons[int(len(patrons) * 0.8):]

def separa_dades(dades):
    """Retorna dues llistes: una amb les dades d'entrada i l'altra amb les dades de sortida."""
    x = []
    y = []
    for dada in dades:
        x.append(dada[:-2])
        y.append(dada[-2:])
    return x, y

x_train, y_train = separa_dades(patrons_entrenament)
x_test, y_test = separa_dades(patrons_validacio)

y_train

[[0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.2, 0.4],
 [0.5, 0.0],
 [0.5, 0.0],
 [0.5, 0.0],

## Crear la xarxa de control del robot

Crearem una xarxa neuronal amb tres capes: entrada, oculta i sortida. Les entrades representaran els sensors del robot i les sortides representaran les accions del robot. La capa oculta ajudarà a transformar els sensors en les accions adequades.

La xarxa ha de tenir tantes entrades com hàgiu utilitzat en el vostre algorisme. Per exemple, en el meu programa, he utilitzat 2 sonars i un sensor de llum per a detectar l'objectiu. Podeu haver utilitzat un nombre diferent de sonars, i per tant tenir una mida diferent.

Comeceu amb una mida de capa oculta de 5, però podeu experimentar amb això per veure quina mida funciona millor.

La mida de la capa de sortida ha de ser 2, ja que aquesta capa representa les quantitats de traducció i rotació per moure el robot.

Recomanem utilitzar `SimpleNetwork` de la llibreria `aitk.networks` per crear la xarxa neuronal; podem, però, utilitzar qualsevol altra llibreria de xarxes neuronals que vulguem.

In [40]:
robot_net = # Crea la xarxa neuronal

El següent pas és entrenar la xarxa neuronal. Degut a la naturalesa de les dades, és probable que necessitem més èpoques d'entrenament que en altres tasques.

L'objectiu serà arrivar a un `accuracy` de més del 80%.

In [60]:
# Entrena la xarxa neuronal

Epoch 15000/15000 loss: 0.014009297825396061 - tolerance_accuracy: 0.631130039691925 - val_loss: 0.014009258709847927 - val_tolerance_accuracy: 0.631130039691925


Fem una prova amb la xarxa neuronal amb les dades d'entrenament i veiem com es comporta.

1/1 [==============================] - 0s 126ms/step


array([0.26664203, 0.03349305], dtype=float32)

## Crear la funció per controlar el robot

Un cop la xarxa neuronal estigui entrenada, crearem una funció que utilitzi la xarxa per controlar el robot. Aquesta funció hauria de prendre les dades dels sensors com a entrada i retornar les comandes per al robot.

In [57]:
def network_driver(robot):
    """Utilitza la xarxa neuronal per a controlar el robot."""

Finalment farem una prova amb el robot per veure com es comporta. 

In [15]:
world = bots.World(240, 150)

world.add_wall("blue", 0, 0, 50, 50)
world.add_wall("blue", 75, 200, 125, 150)
world.add_wall("blue", 150, 0, 200, 50)
world.add_wall("blue", 150, 150, 200, 100)
world.add_wall("blue", 0, 100, 50, 150)

world.add_wall("blue", 100, 25, 105, 125)
world.add_wall("blue", 100, 70, 120, 75)

world.add_bulb("yellow", 125, 130, 1, 40)

robot = bots.Scribbler(x=30, y=80, a=90)
robot.add_device(bots.RangeSensor(width=45,max=20,name="front"))
robot.add_device(bots.RangeSensor(width=45,max=20,position=(6,-6),
                                         a=90,name="left"))
robot.add_device(bots.LightSensor(position=(6, 0), name="light"))

world.add_robot(robot)

world.watch()

Random seed set to: 3301372


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

: 

In [61]:
world.reset()
world.seconds(90, [network_driver], real_time=False)

Using random seed: 4133351


  0%|          | 0/900 [00:00<?, ?it/s]

[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[1. 1. 0.]
[-0.1105957   0.35740483]
[0.92013389 1.         0.        ]
[0.06357467 0.29292658]
[0.86552539 1.         0.        ]
[0.1829164  0.23711756]
[0.82956132 1.         0.        ]
[0.25727916 0.19642422]
[0.80794592 1.         0.        ]
[0.2993475  0.17092903]
[0.79763973 1.         0.        ]
[0.3185745  0.15858744]
[0.79259021 1.         0.        ]
[0.32778412 0.15250956]
[0.78972049 1.         0.        ]
[0.3329539  0.14904754]
[0.78706699 1.         0.        ]
[0.33769196 0.14584354]
[0.78462254 0.98114434 0.        ]
[0.32868096 0.08152583]
[0.78820519 0.94727498 0.        ]
[ 0.2991652  -0.02210019]
[0.79848608 0.91800695 0.        ]
[ 0

True

In [62]:
# Guardem la xarxa
robot_net.save("robot")

INFO:tensorflow:Assets written to: robot/assets


INFO:tensorflow:Assets written to: robot/assets


## Preguntes a respondre

1. Cal algún tipus de memòria per aconseguir que el robot arribi a l'objectiu? Per què?
   
2. Quin percentatge de les dades d'entrenament fan que el robot es mogui cap endavant, cap a l'esquerra i cap a la dreta?

3. Quina mida de capa oculta us ha funcionat millor?

4. El funcionament del robot controlat per la xarxa neuronal és millor, pitjor o igual que el funcionament del robot controlat pel vostre algorisme? Podríeu explicar per què?

5. Quin mecanisme se t'acudeix per millorar el funcionament del robot controlat per la xarxa neuronal?

## Conclusions

En aquesta pràctica hem après a controlar un robot utilitzant una xarxa neuronal. Hem vist com entrenar la xarxa neuronal amb dades de sensors i comandes i com utilitzar la xarxa per controlar el robot. Hem vist que la xarxa neuronal pot ser una eina potent per controlar robots, però que pot ser necessari un temps d'entrenament més llarg que amb altres mètodes. També hem vist que és important balancejar les dades d'entrenament perquè la xarxa neuronal aprengui bé.

A vegades pot no ser fàcil aconseguir les dades d'entrenament necessàries per aconseguir que la xarxa neuronal funcioni bé. En aquest cas, hem utilitzat dades simulades, però en un entorn real, podria ser més complicat.

En la pròxima pràctica, veurem com entrenar una xarxa neuronal per controlar un robot sense necessitat de dades d'entrenament.